# Kneser-Ney smoothingによるn-gram言語モデルを使った句点挿入(生文デモ)
##  [KenLM](https://github.com/kpu/kenlm) で学習済みのモデル(*.klm)を読み込み

In [ ]:
import kenlm
LM = 'data/3gram.100K.klm'
model = kenlm.Model(LM)
print('{0}-gram model'.format(model.order))

## 入力を分かち書き

In [ ]:
from preprocess import WordDivider

def wakati(line):
    wd = WordDivider(is_normalize=True)
    words = wd.extract_words(line, with_pos=False)
    return ' '.join(words)

# 句読点挿入モデル
## 各位置で句点を挿入する/しないで文全体の尤度比較し、尤度が上がれば句点挿入

In [ ]:
# windowはngramのnにあわせる
def punkt_insert(sentence, punkt='。', window=3, debug=False):
    words = sentence.split()
    pnkt_pos = []
    for i in range(len(words)-window+1):
        prob = model.score(' '.join(words))
        prob_p = model.score(' '.join(words[:i+window] + [punkt] + words[i+window:]))
        if prob_p > prob:
            if debug:
                print(prob_p, prob, words[i:i+window])
            pnkt_pos.append(i+window)
    return pnkt_pos

def punkt_decode(sentence, punkt_pos, punkt='。'):
    words = sentence.split()
    pnkt_sent = []
    for i in range(len(words)):
        pnkt_sent.append(words[i])
        if i+1 in punkt_pos:
            pnkt_sent.append(punkt)
    return ' '.join(pnkt_sent)

# 句読点付き文から句点挿入位置を抽出
def extract_positions(sentence, punkt='。'):
    PUNKTS = ['、', '。']
    true_positions = []
    c_pos = 0
    for w in sentence.split(' '):
        if w in PUNKTS:
            if w == punkt:
                true_positions.append(c_pos)
        else:
            c_pos += 1
    return true_positions

# 句読点無し文に対する句点挿入位置の予測結果から精度と再現率を計算
def prf(punkt_pred, punkt_true):
    tp = set.intersection(punkt_pred, punkt_true)      
    precision = len(tp) / len(punkt_true) if len(punkt_true) > 0 else 0
    recall = len(tp) / len(punkt_pred) if len(punkt_pred) > 0 else 0
    fval = 2 / (1/precision + 1/recall) if precision and recall else 0.
    return precision, recall, fval

## 句読点なし文に句読点を挿入

In [ ]:
sentence_raw = '今日 も いい だ 天気 明日 の 天気 も 晴れ らしい'

In [ ]:
words = wakati(sentence_raw)
punkt_position = punkt_insert(words)
sentence_pred = punkt_decode(punkt_position)
print(sentence_pred)

### 句読点付き正解文と比較し、精度・再現率・F値を算出

In [ ]:
sentence_true = '今日 も 、 いい 天気 だ 。 明日 の 天気 も 晴れ らしい 。' 
p,r,f = prf(extract_positions(sentence_pred), extract_positions(sentence_true))
print('Precision:{0:.5g}, Recall:{1:.5g}, F-value:{2:.5g}'.format(p,r,f))